In [37]:
%load_ext autoreload
%autoreload 2
import pandas as pd

from scraping_functions import get_table_from_game_comments, get_weeks_in_year, get_games_in_week
from parsing_functions import (get_punt_plays_from_df, get_team_punters, 
                               assign_team_to_punts, get_losing_team)
from scoring_functions import (get_yrds_to_endzone, get_field_position_score,
                               get_yrds_to_go_score, get_gamescore_multilpier, 
                               get_gameclock_multiplier, get_surrender_index)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
baseline_punt_df = pd.DataFrame()
error_games_base = []
baseline_years = [2010, 2011, 2012, 2013, 2014]
for year in baseline_years:
    week_urls = get_weeks_in_year(year)
    for week_url in week_urls:
        week = int(week_url.split('_')[1].split('.')[0])
        print(f'{year}, {week}')
        if year <2021:
            is_playoffs = week > 18
        else:
            is_playoffs = week > 17
            
        game_urls = get_games_in_week(week_url)
        
        for game_url in game_urls:
            try:
                pbp_df = get_table_from_game_comments(game_url, 'pbp')
                losing_team = get_losing_team(pbp_df)
                punt_df = get_punt_plays_from_df(pbp_df)
                punters_df = get_table_from_game_comments(game_url, 'kicking', header_row=1)
                punt_df = assign_team_to_punts(punt_df, punters_df)
                punt_df = get_yrds_to_endzone(punt_df)
                punt_df = get_field_position_score(punt_df)
                punt_df = get_yrds_to_go_score(punt_df)
                punt_df = get_gamescore_multilpier(punt_df)
                punt_df = get_gameclock_multiplier(punt_df, year, is_playoffs )
                punt_df = get_surrender_index(punt_df)
            except ValueError:
                error_games_base.append(game_url)
                continue
            baseline_punt_df = baseline_punt_df.append(punt_df)


2010, 20
2010, 13
2010, 2
2010, 17
2010, 21
2010, 5
2010, 15
2010, 19
2010, 16
2010, 9
2010, 8
2010, 11
2010, 14
2010, 1
2010, 7
2010, 4
2010, 6
2010, 10
2010, 12
2010, 18
2010, 3
2011, 9
2011, 1
2011, 13
2011, 6
2011, 14
2011, 7
2011, 21
2011, 11
2011, 19
2011, 5
2011, 10
2011, 8
2011, 17
2011, 18
2011, 16
2011, 20
2011, 2
2011, 15
2011, 3
2011, 12
2011, 4
2012, 12
2012, 3
2012, 11
2012, 13
2012, 14
2012, 1
2012, 9
2012, 15
2012, 6
2012, 18
2012, 7
2012, 19
2012, 16
2012, 10
2012, 4
2012, 8
2012, 20
2012, 21
2012, 17
2012, 5
2012, 2
2013, 6
2013, 12
2013, 13
2013, 11
2013, 9
2013, 18
2013, 4
2013, 10
2013, 15
2013, 19
2013, 17
2013, 16
2013, 3
2013, 8
2013, 21
2013, 7
2013, 5
2013, 20
2013, 14
2013, 2
2013, 1
2014, 13
2014, 10
2014, 12
2014, 11
2014, 17
2014, 16
2014, 5
2014, 9
2014, 18
2014, 4
2014, 3
2014, 8
2014, 21
2014, 7
2014, 1
2014, 14
2014, 2
2014, 19
2014, 15
2014, 20
2014, 6


In [39]:
print(f'{len(error_games_base)} could not be parsed')

55 could not be parsed


In [40]:
cowardly_punt_threshold = baseline_punt_df.surrender_index.quantile(0.9)

In [41]:
games = 0
losing_games = 0
losing_games_with_cowardly_punts = 0
games_with_cowardly_punts = 0
error_games_test = []

test_years = [2015, 2016, 2017, 2018, 2019]
for year in test_years:
    week_urls = get_weeks_in_year(year)
    for week_url in week_urls:
        week = int(week_url.split('_')[1].split('.')[0])
        print(f'{year}, {week}')
        if year <2021:
            is_playoffs = week > 18
        else:
            is_playoffs = week > 17
            
        game_urls = get_games_in_week(week_url)
        
        for game_url in game_urls:
            try:
                pbp_df = get_table_from_game_comments(game_url, 'pbp')
                losing_team = get_losing_team(pbp_df)
                punt_df = get_punt_plays_from_df(pbp_df)
                punters_df = get_table_from_game_comments(game_url, 'kicking', header_row=1)
                punt_df = assign_team_to_punts(punt_df, punters_df)
                punt_df = get_yrds_to_endzone(punt_df)
                punt_df = get_field_position_score(punt_df)
                punt_df = get_yrds_to_go_score(punt_df)
                punt_df = get_gamescore_multilpier(punt_df)
                punt_df = get_gameclock_multiplier(punt_df, year, is_playoffs )
                punt_df = get_surrender_index(punt_df)
            except ValueError:
                error_games_test.append(game_url)
                continue
            
            cowardly_punts = punt_df[punt_df['surrender_index']>cowardly_punt_threshold]
            
            # we're counting team-games here, so each game counts
            # once for each time
            games += 2
            if losing_team is not None:
                losing_games += 1
                
            if len(cowardly_punts)>0:
                games_with_cowardly_punts += len(cowardly_punts['Tm'].unique())
                
                if losing_team in cowardly_punts['Tm'].unique():
                    losing_games_with_cowardly_punts += 1
            


2015, 12
2015, 3
2015, 17
2015, 21
2015, 9
2015, 8
2015, 6
2015, 11
2015, 1
2015, 4
2015, 16
2015, 20
2015, 5
2015, 14
2015, 15
2015, 2
2015, 7
2015, 19
2015, 18
2015, 13
2015, 10
2016, 18
2016, 8
2016, 9
2016, 3
2016, 11
2016, 2
2016, 14
2016, 4
2016, 12
2016, 1
2016, 5
2016, 16
2016, 7
2016, 20
2016, 19
2016, 21
2016, 17
2016, 15
2016, 10
2016, 13
2016, 6
2017, 16
2017, 19
2017, 7
2017, 12
2017, 14
2017, 17
2017, 21
2017, 10
2017, 4
2017, 15
2017, 8
2017, 9
2017, 1
2017, 6
2017, 18
2017, 20
2017, 5
2017, 2
2017, 13
2017, 11
2017, 3
2018, 5
2018, 1
2018, 10
2018, 19
2018, 20
2018, 18
2018, 21
2018, 2
2018, 3
2018, 6
2018, 12
2018, 14
2018, 13
2018, 4
2018, 7
2018, 15
2018, 17
2018, 16
2018, 9
2018, 11
2018, 8
2019, 20
2019, 10
2019, 18
2019, 9
2019, 8
2019, 4
2019, 1
2019, 7
2019, 15
2019, 6
2019, 12
2019, 17
2019, 2
2019, 21
2019, 11
2019, 3
2019, 14
2019, 16
2019, 5
2019, 13
2019, 19


In [42]:
print(f'{len(error_games_test)} could not be parsed')

20 could not be parsed


In [43]:
games 

2608

In [44]:
losing_games 

1297

In [45]:
losing_games_with_cowardly_punts 

449

In [46]:
games_with_cowardly_punts 

786

In [47]:
baseline_punt_df.surrender_index.quantile(0.9)

7.546178307520002

In [48]:
baseline_punt_df.surrender_index.quantile(0)

0.2

In [49]:
baseline_punt_df.surrender_index.quantile(0.5)

1.2

In [50]:
baseline_punt_df.surrender_index.quantile(0.5)

1.2

In [51]:
(449.0/1297.0)*(1297/2608)/(786/2608.0)

0.5712468193384224